# Weight of evidence

**Weight of Evidence (WOE)** dá indícios sobre a capacidade de uma covariável conseguir distinguir bons e maus clientes para a variável resposta. Sua fórmula é dada por:

$$\text{WOE} = \ln \left[ \frac{\text{\% de clientes bons}}{\text{\% de clientes ruins}} \right]$$

- $\text{WOE} > 0$ indica que a distribuição de bons é maior que a de maus;
- $\text{WOE} < 0$ indica que a distribuição de maus é maior do que de bons.

A medida surgiu em modelos de Regressão Logística com objetivo de auxiliar os analistas em categorizar covariáveis contínuas, permitindo que novas covariáveis possam pertencer ao modelo mesmo que a hipótese de colinearidade não seja satisfeita.

Para isto;

1. Para variáveis contínuas separamos-as em 10 a 20 categorias;
2. Calculamos o número de eventos e não eventos em cada grupo;
3. Calculamos a % de eventos e % de não eventos em cada grupo;
4. Calculamos o WOE em cada grupo

Para variáveis categóricas a divisão claramente não é necessária, com a medida sendo auxiliar na hora de agruparmos duas categorias com mesmo comportamento.

<center>

| Categorias | Não-eventos |  Eventos | % Não-eventos | % Eventos |   WOE   |   IV   |
|:----------:|:-----------:|:--------:|:-------------:|:---------:|:-------:|:------:|
|    0-50    |     197     |    20    |     5.4\%     |   5.9\%   | -0.0952 | 0.0005 |
|   51-100   |     450     |    34    |     12.3\%    |   10.1\%  |  0.2002 | 0.0045 |
|   101-150  |     492     |    39    |     13.4\%    |   11.5\%  |  0.1522 | 0.0029 |
|   151-200  |     597     |    51    |     16.3\%    |   15.1\%  |  0.0774 | 0.0009 |
|   201-250  |     609     |    54    |     16.6\%    |   16.0\%  |  0.0401 | 0.0003 |
|   251-300  |     582     |    55    |     15.9\%    |   16.3\%  | -0.0236 | 0.0001 |
|   301-350  |     386     |    41    |     10.5\%    |   12.1\%  | -0.1405 | 0.0022 |
|   351-400  |     165     |    23    |     4.5\%     |   6.8\%   | -0.4123 | 0.0095 |
|    >401    |     184     |    21    |     5.0\%     |   6.2\%   | -0.2123 | 0.0025 |
|            |  **Total**  | **3662** |    **338**    |           |         |        |

</center>

## Regras associadas ao WOE

1. Cada categoria deve ter pelo menos 5% de observações
2. Cada categoria deve ter uma quantidade não nula de eventos e não eventos;
3. WOE deve ser distinto para cada categoria, com grupos similares sendo recomendado o agrupamento;
4. Valores NA são analisados separadamente.

## Lidando com categorias sem Eventos/Não-eventos

Se uma categoria em particular não possuir eventos/não-eventos, podemos utilizar o WOE ajustado para ignorar este problema:

$$\text{WOE}_{\text{Ajustado}} = \ln \left[ \frac{\text{\% de clientes bons}+0.5}{\text{\% de clientes ruins}+0.5} \right]$$

Dentre as vantagens de utilizarmos Weight of Evidence, temos:

1. Capacidade de tratar outliers devido a categorização de variáveis lineares;
2. Valores NA podem ser tratados separadamente, facilitando a explicabilidade;
3. Auxilia na construção de uma relação linear entre as covariáveis e a variável resposta, facilitando a interpretação do log-odds.

# Information Value

**Information value (IV)** é uma técnica para selecionar as variáveis mais importantes em um modelo preditivo. Sua fórmula é dada por:

$$
\text{IV} = \sum(\text{\% de nao eventos} - \text{\% de eventos}) \cdot \text{WOE}
$$

A seguinte tabela fornece bons indícios de como interpretar valores de Information Value:

<center>

| **Information Value** |           **Descrição**           |
|:---------------------:|:---------------------------------:|
|         < 0.02        |     Irrelevante para predição     |
|      0.02 até 0.1     |       Poder preditivo baixo       |
|      0.1 até 0.3      |       Poder preditivo médio       |
|      0.3 até 0.5      |       Poder preditivo forte       |
|         > 0.5         | Poder preditivo suspeito (checar) |

</center>

Questões importantes:

1. Information Value aumenta conforme o número de categorias aumenta, dessa forma é necessário atenção para o modelo possuir parcimônia;
2. Não é uma métrica tão relevante quando utilizamos outros modelos além do de regressão logística

# Exemplos Computacionais

In [1]:
import pandas as pd
import numpy as np
mydata = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

In [ ]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d0 = d0.astype({"x": str})
        d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Non-Events']/d['% of Events'])
        d['IV'] = d['WoE'] * (d['% of Non-Events']-d['% of Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF